# ***Text vectorization***

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
from scipy.sparse import hstack
import os , pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm

import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)
  

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.chdir(mount +'/MyDrive/RBC_V4')

In [ ]:
#df = pd.read_csv("/content/gdrive/MyDrive/W2VEC_PullRequest2/data_with_preprocess_2.csv")
df = pd.read_csv(mount +'/MyDrive/RBC_V4/W2V_VR4.csv', delimiter=';', encoding='cp437')
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))
print(df.shape)
df.head(2)

(3328, 6)


,id,qid1,question1,qid2,question2,is_duplicate
0,0,14082,providerfastly Make requestcondition optional ...,14103,providerfastly Changes setting conditionals to...,1
1,1,6533,add `validatecredentials` option to provideraw...,7874,Skip IAMSTS validation and metadata check Skip...,1


# ***For the ease of computation we will sample only 100k points***

In [ ]:
#sampling 100k
#df = df.sample(n=100000,random_state=40)

In [ ]:
#changing columns to numeric type
num_cols = df.drop(columns=['id', 'qid1', 'qid2', 'question1', 'question2']).columns
for i in num_cols:
    df[i] = df[i].apply(pd.to_numeric)

In [ ]:
y = df['is_duplicate']
X = df[df.drop(columns=['id', 'qid1', 'qid2','is_duplicate']).columns.tolist()]
print(X.shape)
print(y.shape)

(3328, 2)
(3328,)


In [ ]:
X_train = X[:2662]
y_train = y[:2662]
X_test = X[2662:]
y_test = y[2662:]

# ***Train test split***

In [ ]:
#X_train,X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2,random_state=100)
print("Number of data points in train data :",X_train.shape)
print("Number of data points in test data :",X_test.shape)

Number of data points in train data : (2662, 2)
Number of data points in test data : (666, 2)


In [ ]:
y_test

2662    1
2663    1
2664    1
2665    1
2666    1
2667    1
2668    1
2669    1
2670    1
2671    1
2672    1
2673    1
2674    1
2675    1
2676    1
2677    1
2678    1
2679    1
2680    1
2681    1
2682    1
2683    1
2684    1
2685    1
2686    1
2687    1
2688    1
2689    1
2690    1
2691    1
2692    1
2693    1
2694    1
2695    1
2696    1
2697    1
2698    1
2699    1
2700    1
2701    1
2702    1
2703    1
2704    1
2705    1
2706    1
2707    1
2708    1
2709    1
2710    1
2711    1
2712    1
2713    1
2714    1
2715    1
2716    1
2717    1
2718    1
2719    1
2720    1
2721    1
2722    1
2723    1
2724    1
2725    1
2726    1
2727    1
2728    1
2729    1
2730    1
2731    1
2732    1
2733    1
2734    1
2735    1
2736    1
2737    1
2738    1
2739    1
2740    1
2741    1
2742    1
2743    1
2744    1
2745    1
2746    1
2747    1
2748    1
2749    1
2750    1
2751    1
2752    1
2753    1
2754    1
2755    1
2756    1
2757    1
2758    1
2759    1
2760    1
2761    1


In [ ]:
y_train

0       1
1       1
2       1
3       1
4       1
       ..
2657    0
2658    0
2659    0
2660    0
2661    0
Name: is_duplicate, Length: 2662, dtype: int64

In [ ]:
X_test

,question1,question2
2662,fixed kubelet typo Where is a typo in contribi...,Fix typo in systemd kubelet apiservers flag. C...
2663,Allow kubecfg to pull urls via c Only valid 'h...,Add support to fetch config file from network ...
2664,Update kubeadm token to work as expected What ...,fix kubeadm token create allow run kubeadm ex ...
2665,Cleanup to client initialization in Kubelet Wa...,Create client outside of Kubelet object. Add a...
2666,custom resource DeRegistration testcase deadsk...,kubeapiextensionsserver Fix potential SEGV wit...
2667,"Added version file. This merge is for , , , wh...",delete ubuntu kubeup Remove deprecated ubuntu ...
2668,Automated cherry pick of Cherry pick of on re...,Automated cherry pick of Cherry pick of on r...
2669,Hyperkube don't parse args for all servers Kub...,Fix hyperkube flag parsing Hyperkube flag pars...
2670,Fix gendocs on master master seems to be broke...,"Run hackrungendocs.sh to fix issue, otherwise ..."
2671,Updated wording from minion to node in cluster...,Rename minionnode for Vagrant provider for The...


In [ ]:
pd.set_option('display.max_rows',X_test.shape[0]+1)
X_test

,question1,question2
2662,fixed kubelet typo Where is a typo in contribi...,Fix typo in systemd kubelet apiservers flag. C...
2663,Allow kubecfg to pull urls via c Only valid 'h...,Add support to fetch config file from network ...
2664,Update kubeadm token to work as expected What ...,fix kubeadm token create allow run kubeadm ex ...
2665,Cleanup to client initialization in Kubelet Wa...,Create client outside of Kubelet object. Add a...
2666,custom resource DeRegistration testcase deadsk...,kubeapiextensionsserver Fix potential SEGV wit...
2667,"Added version file. This merge is for , , , wh...",delete ubuntu kubeup Remove deprecated ubuntu ...
2668,Automated cherry pick of Cherry pick of on re...,Automated cherry pick of Cherry pick of on r...
2669,Hyperkube don't parse args for all servers Kub...,Fix hyperkube flag parsing Hyperkube flag pars...
2670,Fix gendocs on master master seems to be broke...,"Run hackrungendocs.sh to fix issue, otherwise ..."
2671,Updated wording from minion to node in cluster...,Rename minionnode for Vagrant provider for The...


In [ ]:
X_train

,question1,question2
0,providerfastly Make requestcondition optional ...,providerfastly Changes setting conditionals to...
1,add `validatecredentials` option to provideraw...,Skip IAMSTS validation and metadata check Skip...
2,"Update environments.html.md kind of confusing,...",Documentation Update Fix a code sample that us...
3,Remove redundant const from command package Re...,command Remove second DefaultDataDirectory con...
4,provideraws Fixing IAM data source policy gene...,provideraws Normalize IAM policy documents Fix...
...,...,...
2657,Fix panic when starting container with invalid...,Fix cpu spin waiting for log write events This...
2658,TestCgroupNamespacesRunOlderClient support cgr...,overlay fix unmounting layer without merge dir...
2659,builder produce error when using unsupported D...,fix incorect login client credential save when...
2660,Handle long log messages correctly on SizedLog...,Reset stdin config before running build action...


# ***Handling text data***

We have already cleaned the text data. Now we have to vectorize it. We mainly used 2 approached.

1. TFIDF
2. TFIDF weighted glove vectorization

# ***TFIDF***

In [ ]:
tfidf_vectorizer1 = TfidfVectorizer(lowercase=False,max_features= 150)
trainqs1_tfidf = tfidf_vectorizer1.fit_transform(X_train['question1'])
testqs1_tfidf  = tfidf_vectorizer1.transform(X_test['question1'])
print(trainqs1_tfidf.shape)
print(testqs1_tfidf.shape)

(2662, 150)
(666, 150)


In [ ]:
tfidf_vectorizer2 = TfidfVectorizer(lowercase=False,max_features= 150)
train_qs2_tfidf = tfidf_vectorizer2.fit_transform(X_train['question2'])
test_qs2_tfidf  = tfidf_vectorizer2.transform(X_test['question2'])
print(train_qs2_tfidf.shape)
print(test_qs2_tfidf.shape)

(2662, 150)
(666, 150)


In [ ]:
#Now we will hstack both the vectors
tfidf_train_vec = hstack((trainqs1_tfidf,train_qs2_tfidf))
tfidf_test_vec = hstack((testqs1_tfidf,test_qs2_tfidf)) 
print("train data shape",tfidf_train_vec.shape)
print("Test data shape ",tfidf_test_vec.shape)

train data shape (2662, 300)
Test data shape  (666, 300)


In [ ]:
# selecting other features
train_df = X_train.drop(columns=['question1', 'question2'])
test_df = X_test.drop(columns=['question1', 'question2'])

In [ ]:
#we need to convert our data with features into sparse matrix so that we can combine our feature matrix and and tfidf vectors 
import scipy
train_sparse = scipy.sparse.csr_matrix(train_df)
test_sparse = scipy.sparse.csr_matrix(test_df)

In [ ]:
# Now combining our tfidf and features into one 
tfidf_X_tr = hstack((train_sparse,tfidf_train_vec))
tfidf_X_test = hstack((test_sparse,tfidf_test_vec))
print("train data shape",tfidf_X_tr.shape)
print("Test data shape ",tfidf_X_test.shape)

train data shape (2662, 300)
Test data shape  (666, 300)


In [ ]:
#saving tfidf vectors
pickle.dump(tfidf_train_vec, open("/content/gdrive/MyDrive/RBC_V4/tfidf_X_tr","wb"))
pickle.dump(tfidf_test_vec, open("/content/gdrive/MyDrive/RBC_V4/tfidf_X_test","wb"))

# ***TFIDF Weighted Glove Vectors***

In [ ]:
# use spacy embedding
# run this from a normal command line
# !python -m spacy download en_core_web_md

In [ ]:
# merge texts
questions = list(X_train['question1']) + list(X_train['question2'])
tfidf = TfidfVectorizer(lowercase=False)
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
# Load the spacy model that you have installed
import en_core_web_sm
nlp = en_core_web_sm.load()

# each vector will be of length 94..
doc = nlp("This is some text that I am processing with Spacy")
#example
doc[3].vector.shape

(96,)

In [ ]:
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_train['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 96])
    for i,word1 in enumerate(doc1):
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1[i] += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)

X_train_glove_q1 = vecs1

100%|██████████| 2662/2662 [01:01<00:00, 42.94it/s]


In [ ]:
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_train['question2'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 96])
    for i,word1 in enumerate(doc1):
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1[i] += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)

X_train_glove_q2 = vecs1

100%|██████████| 2662/2662 [00:58<00:00, 45.44it/s]


In [ ]:
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_test['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 96])
    for i,word1 in enumerate(doc1):
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1[i] += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)

X_test_glove_q1 = vecs1

100%|██████████| 666/666 [00:15<00:00, 43.72it/s]


In [ ]:
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_test['question2'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 96])
    for i,word1 in enumerate(doc1):
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1[i] += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)

X_test_glove_q2 = vecs1

100%|██████████| 666/666 [00:14<00:00, 44.99it/s]


In [ ]:
X_train['q1_glove'] = X_train_glove_q1
X_train['q2_glove'] = X_train_glove_q2
X_test['q1_glove'] = X_test_glove_q1
X_test['q2_glove'] = X_test_glove_q2

In [ ]:
train_glove = np.concatenate([np.array(X_train_glove_q1),np.array(X_train_glove_q2)],axis=1)
test_glove = np.concatenate([np.array(X_test_glove_q1),np.array(X_test_glove_q2)],axis=1)
train_glove.shape

(2662, 192)

In [ ]:
glove_train_df = pd.DataFrame(train_glove,columns=[f'g_{i}' for i in range(train_glove.shape[1])])
glove_test_df = pd.DataFrame(test_glove,columns=[f'g_{i}' for i in range(test_glove.shape[1])])
glove_train_df.head()

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9,...,g_182,g_183,g_184,g_185,g_186,g_187,g_188,g_189,g_190,g_191
0,2.144256,0.059182,-4.144643,-0.622516,3.115685,-2.216018,5.296974,2.016154,3.065483,6.017498,...,0.340742,2.075201,-1.607179,4.144705,0.496864,-2.950733,2.629562,-1.680025,2.643243,8.849958
1,0.319335,0.845038,-2.139519,-1.870514,4.791848,2.084213,3.031052,0.467578,5.935488,6.970626,...,0.310578,2.028433,-1.127390,3.834930,2.459218,-1.576828,-2.811744,3.016457,5.939093,3.799270
2,3.043343,1.969943,-0.224947,-1.830269,4.697424,-0.312517,2.763675,2.256241,-0.420380,4.242164,...,-2.055058,1.406997,-0.885238,4.082704,0.908822,0.017984,0.016431,3.460400,2.611437,3.959778
3,4.443172,-1.266115,-0.641468,2.520198,9.988480,3.266514,4.659507,3.561577,7.662127,6.538033,...,0.535373,5.169972,-2.534284,-0.091309,-0.005965,0.679608,-5.611737,4.142711,6.679124,5.980587
4,2.568849,0.469732,-2.380582,-0.533666,2.581121,0.709937,3.577694,1.949230,4.896126,7.856310,...,0.157204,-0.785157,-2.282226,2.875941,-2.539889,-2.985559,1.002034,-1.949714,5.443133,7.053291


In [ ]:
X_train = X_train.drop(columns=['question1','question2']).reset_index(drop=True)
X_test = X_test.drop(columns=['question1','question2']).reset_index(drop=True)
print(X_train.shape)
print(X_test.shape)

(2662, 2)
(666, 2)


In [ ]:
# concatenating
X_train_d = pd.concat([X_train,glove_train_df],axis=1)
X_test_d = pd.concat([X_test,glove_test_df],axis=1)
print(X_train_d.shape)
print(X_test_d.shape)

(2662, 194)
(666, 194)


In [ ]:
X_train_d.to_csv('/content/gdrive/MyDrive/RBC_V4/train_data.csv',index=False)
X_test_d.to_csv('/content/gdrive/MyDrive/RBC_V4/test_data.csv',index=False)

In [ ]:
y_train.to_csv('/content/gdrive/MyDrive/RBC_V4/train_y.csv',index=False)
y_test.to_csv('/content/gdrive/MyDrive/RBC_V4/test_y.csv',index=False)